In [1]:
using NBInclude
@nbinclude("../heuristics/randommap.ipynb")
@nbinclude("../heuristics/localsearch.ipynb")

In [2]:
function read_file(genomesize::Int)
    filename = "db/rep_db" * string(genomesize) * ".txt"
    content = read(filename, String)
    genome_pairs = split(content, "|")

    return genome_pairs
end 


read_file (generic function with 1 method)

In [3]:
function evaluate_one_pair_rm(genome_pair::String, genomesize::Int, dcjs::Vector{Int}) 
    g_pair = split(genome_pair, ";")

    src = string(g_pair[1])
    target = string(g_pair[2])

    num_maps = 10 * genomesize

    _, map_to_dedupstr_dcjdist = randommap(src, target, num_maps, "none")
    dcjdists = map(t -> t[2], values(map_to_dedupstr_dcjdist))
    mindcj = minimum(dcjdists)
    # print(mindcj, " ")

    push!(dcjs, mindcj)
end 


function evaluate_one_rep_rm(genomesize::Int)
    genome_pairs = read_file(genomesize::Int)

    dcjs = Vector{Int}()
    times = Vector{Float64}()

    i = 1
    for g_pair in genome_pairs 
        println(genomesize, " ", i)
        
        time_taken = @elapsed begin
            evaluate_one_pair_rm(string(g_pair), genomesize, dcjs)
        end  

        push!(times, time_taken)
        # println(time_taken)
        i += 1
    end 

    return mean(dcjs), mean(times)
end 

# i = 150
# dcj, time = evaluate_rep_rm(i)

evaluate_one_rep_rm (generic function with 1 method)

In [4]:
# str = "," * string(dcj) * " " * string(time)

# filename = "tmp_rm" * ".txt"

# open(filename, "a") do file
#     write(file, str)
# end

In [5]:
# loop through all REP sets (strings of size 50 - 500)
# but if it bugs out, lose all data 
# function evaluate_rm()
#     times = []
#     dists = Vector{Float64}() 
#     for i in 50:50:500 
#         avgdcj, time = evaluate_one_rep_rm(i)
#         push!(times, time)
#         push!(dists, avgdcj)
#     end 

#     return dists, times
# end 

# dists, times = evaluate_rm()



In [6]:
# using CSV
# using DataFrames

# function write_to_csv(heuristic::String, dists::Vector{Int}, times::Vector{Int})
#     avgtime = average(times)

#     data = DataFrame(Heuristic = [heuristic], Time= [avgtime], str50 = [dists[1]], str100 = [dists[2]], str150 = [avgdist[3]], str200 = [avgdist[4]], str250 = [avgdist[5]], str300 = [avgdist[6]], str350 = [avgdist[7]], str400 = [avgdist[8]], str450 = [avgdist[9]], str500 = [avgdist[10]])

#     file_path = "output.csv"

#     if isfile(file_path)
#         # append to existing data 
#         existing_data = CSV.read(file_path, DataFrame)
#         updated_data = vcat(existing_data, data)
#         CSV.write(file_path, updated_data)
#     else
#         CSV.write(file_path, data)
        
#     end 
# end 

# heuristic = "RM"
# write_to_csv(heuristic, dists, times)


In [7]:
# rand maps = 60; neighbors = 15 

function evaluate_one_pair_ls(genome_pair::String, genomesize::Int, dcjs::Vector{Int}, mcn::Bool, total_maps::Int, rand_maps::Int, max_neighbors::Int)
    g_pair = split(genome_pair, ";")

    src = string(g_pair[1])
    target = string(g_pair[2])

    
    
    global_min_dcj, _, _, _, _ = localsearch(src, target, total_maps, rand_maps, max_neighbors, "none", mcn, Vector())

    push!(dcjs, global_min_dcj)
end 


function evaluate_one_rep_ls(genomesize::Int, mcn::Bool, genome_pairs::Vector{SubString{String}}, rand_maps::Int, max_neighbors::Int)
    dcjs = Vector{Int}()
    times = Vector{Float64}()

    for g_pair in genome_pairs
        total_maps = 10 * genomesize
        
        time_taken = @elapsed begin
            evaluate_one_pair_ls(string(g_pair), genomesize, dcjs, mcn, total_maps, rand_maps, max_neighbors)
        end  
        push!(times, time_taken)
    end 

    return mean(dcjs), mean(times) 
end 


evaluate_one_rep_ls (generic function with 1 method)

In [8]:
# loop through all REP sets (strings of size 50 - 500)
# but if it bugs out, lose all data 

# function evaluate_ls(mcn::Bool)
#     times = []
#     dists = Vector{Float64}() 
#     for i in 50:50:500 
#         avgdcj, time = evaluate_one_rep_ls(i, mcn)
#         push!(times, time)
#         push!(dists, avgdcj)
#     end 

#     return dists, times
# end 

# mcn = true 
# dists, times = evaluate_ls(mcn)